**Importing the necessary libraries**

In [18]:
import pandas as pd
import numpy as np
import ast

**Preparing the dataset**

In [19]:
df=pd.read_csv('data_jobs_cleaned.csv')
df_us_job_counts=pd.read_csv('1_data_jobs_location_counts.csv')
df['job_location'] = df['job_location'].str.split(',', expand=True)[0].str.strip()
cities_list=df_us_job_counts['job_location'].tolist()
cities_list

['New York', 'Atlanta', 'Chicago', 'Dallas', 'Austin']

**Filtering for the 5 cities**

In [20]:
df_5_cities=df[df['job_location'].isin(cities_list)]

**Transforming the column 'job_skills' from string datatype to a list**

In [21]:
df_5_cities['job_skills']=df_5_cities['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

C:\Users\vbali\AppData\Local\Temp\ipykernel_2992\3733319462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5_cities['job_skills']=df_5_cities['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)


**Filtering for analyst roles**

In [22]:
df_5_cities=df_5_cities[df_5_cities['job_title_short'].str.contains('Analyst')]

**Exploding column 'job_skills' to be able to analyze skills**

In [23]:
df_5_exp=df_5_cities.explode('job_skills')

**Creating a dataframe per city**

In [24]:
skills_AU=df_5_exp[df_5_exp['job_location']=='Austin']['job_skills'].value_counts().head()
skills_AU=skills_AU.reset_index(name='count')
skills_AT=df_5_exp[df_5_exp['job_location']=='Atlanta']['job_skills'].value_counts().head()
skills_AT=skills_AT.reset_index(name='count')
skills_NY=df_5_exp[df_5_exp['job_location']=='New York']['job_skills'].value_counts().head()
skills_NY=skills_NY.reset_index(name='count')
skills_CH=df_5_exp[df_5_exp['job_location']=='Chicago']['job_skills'].value_counts().head()
skills_CH=skills_CH.reset_index(name='count')
skills_DA=df_5_exp[df_5_exp['job_location']=='Dallas']['job_skills'].value_counts().head()
skills_DA=skills_DA.reset_index(name='count')

**Creating containers (a dictionary and a list)** 

In [25]:
cities_dict={}
skills_cities=[]

**Appending the list with the cities' dataframes**

In [26]:
skills_cities.append(skills_AU)
skills_cities.append(skills_AT)
skills_cities.append(skills_NY)
skills_cities.append(skills_CH)
skills_cities.append(skills_DA)
skills_cities

[  job_skills  count
 0        sql    851
 1      excel    664
 2    tableau    585
 3     python    542
 4        sas    294,
   job_skills  count
 0        sql   1295
 1      excel    930
 2    tableau    750
 3     python    653
 4        sas    552,
   job_skills  count
 0        sql   1552
 1      excel   1147
 2     python    916
 3    tableau    907
 4          r    518,
   job_skills  count
 0        sql   1101
 1      excel    873
 2     python    612
 3    tableau    610
 4        sas    420,
   job_skills  count
 0        sql    891
 1      excel    638
 2    tableau    487
 3     python    427
 4   power bi    326]

**Putting the list's content into the dictionary with city names as keys and dataframes as values**

In [27]:
for i in range(5):
    cities_dict[cities_list[i]]=skills_cities[i]

In [28]:
cities_dict

{'New York':   job_skills  count
 0        sql    851
 1      excel    664
 2    tableau    585
 3     python    542
 4        sas    294,
 'Atlanta':   job_skills  count
 0        sql   1295
 1      excel    930
 2    tableau    750
 3     python    653
 4        sas    552,
 'Chicago':   job_skills  count
 0        sql   1552
 1      excel   1147
 2     python    916
 3    tableau    907
 4          r    518,
 'Dallas':   job_skills  count
 0        sql   1101
 1      excel    873
 2     python    612
 3    tableau    610
 4        sas    420,
 'Austin':   job_skills  count
 0        sql    891
 1      excel    638
 2    tableau    487
 3     python    427
 4   power bi    326}

**Creating a merged dataframe for city dataframes and defining the column names**

In [29]:
merged_df = pd.DataFrame(columns=['job_location', 'job_skills', 'count'])

**Concatenating the cities' dataframes**

In [30]:
for loc, df in cities_dict.items():
    df['job_location']=loc
    merged_df=pd.concat([merged_df, df], ignore_index=True)
merged_df

,job_location,job_skills,count
0,New York,sql,851
1,New York,excel,664
2,New York,tableau,585
3,New York,python,542
4,New York,sas,294
5,Atlanta,sql,1295
6,Atlanta,excel,930
7,Atlanta,tableau,750
8,Atlanta,python,653
9,Atlanta,sas,552


**Counting the number of total job postings per city**

In [31]:
jobs_total=df_5_cities.groupby('job_location').size()
jobs_total=jobs_total.reset_index(name='count_total')

**Merging the concatenated dataframe with the 'total' dataframe**

In [32]:
merged_df=merged_df.merge(jobs_total, on='job_location')
merged_df

,job_location,job_skills,count,count_total
0,New York,sql,851,2717
1,New York,excel,664,2717
2,New York,tableau,585,2717
3,New York,python,542,2717
4,New York,sas,294,2717
5,Atlanta,sql,1295,2419
6,Atlanta,excel,930,2419
7,Atlanta,tableau,750,2419
8,Atlanta,python,653,2419
9,Atlanta,sas,552,2419


**Calculating the probability of skills appearing in job postings (per cities)**

In [33]:
merged_df['percentage']=(merged_df['count']/merged_df['count_total']*100).astype(int)
merged_df

,job_location,job_skills,count,count_total,percentage
0,New York,sql,851,2717,31
1,New York,excel,664,2717,24
2,New York,tableau,585,2717,21
3,New York,python,542,2717,19
4,New York,sas,294,2717,10
5,Atlanta,sql,1295,2419,53
6,Atlanta,excel,930,2419,38
7,Atlanta,tableau,750,2419,31
8,Atlanta,python,653,2419,26
9,Atlanta,sas,552,2419,22


**Writing the dataframe into a csv file**

In [34]:
merged_df.to_csv('4_da_top_skills.csv', index=False)